In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import confusion_matrix,mean_squared_error,r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('./Melbourne_housing_FULL.csv')
df

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.79960,144.99840,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.80790,144.99340,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.81140,145.01160,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.80930,144.99440,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,13 Burns St,4,h,1480000.0,PI,Jas,24/02/2018,6.3,3013.0,4.0,1.0,3.0,593.0,NaN,NaN,Maribyrnong City Council,-37.81053,144.88467,Western Metropolitan,6543.0
34853,Yarraville,29A Murray St,2,h,888000.0,SP,Sweeney,24/02/2018,6.3,3013.0,2.0,2.0,1.0,98.0,104.0,2018.0,Maribyrnong City Council,-37.81551,144.88826,Western Metropolitan,6543.0
34854,Yarraville,147A Severn St,2,t,705000.0,S,Jas,24/02/2018,6.3,3013.0,2.0,1.0,2.0,220.0,120.0,2000.0,Maribyrnong City Council,-37.82286,144.87856,Western Metropolitan,6543.0
34855,Yarraville,12/37 Stephen St,3,h,1140000.0,SP,hockingstuart,24/02/2018,6.3,3013.0,NaN,NaN,NaN,NaN,NaN,NaN,Maribyrnong City Council,NaN,NaN,Western Metropolitan,6543.0


In [4]:
df.nunique()

Suburb             351
Address          34009
Rooms               12
Type                 3
Price             2871
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom2            15
Bathroom            11
Car                 15
Landsize          1684
BuildingArea       740
YearBuilt          160
CouncilArea         33
Lattitude        13402
Longtitude       14524
Regionname           8
Propertycount      342
dtype: int64

In [5]:
cols_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount', 
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price']
df = df[cols_to_use]
df

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,4,h,PI,Jas,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,4.0,1.0,3.0,593.0,NaN,1480000.0
34853,Yarraville,2,h,SP,Sweeney,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,2.0,2.0,1.0,98.0,104.0,888000.0
34854,Yarraville,2,t,S,Jas,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,2.0,1.0,2.0,220.0,120.0,705000.0
34855,Yarraville,3,h,SP,hockingstuart,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,NaN,NaN,NaN,NaN,NaN,1140000.0


In [6]:
df.isna().sum()

Suburb               0
Rooms                0
Type                 0
Method               0
SellerG              0
Regionname           3
Propertycount        3
Distance             1
CouncilArea          3
Bedroom2          8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
Price             7610
dtype: int64

In [7]:
# Some feature's missing values can be treated as zero (another class for NA values or absence of that feature)
# like 0 for Propertycount, Bedroom2 will refer to other class of NA values
# like 0 for Car feature will mean that there's no car parking feature with house
cols_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
df[cols_to_fill_zero] = df[cols_to_fill_zero].fillna(0)

df.isna().sum()

Suburb               0
Rooms                0
Type                 0
Method               0
SellerG              0
Regionname           3
Propertycount        0
Distance             0
CouncilArea          3
Bedroom2             0
Bathroom             0
Car                  0
Landsize         11810
BuildingArea     21115
Price             7610
dtype: int64

In [8]:
# other continuous features can be imputed with mean for faster results since our focus is on Reducing overfitting
# using Lasso and Ridge Regression
df['Landsize'] = df['Landsize'].fillna(df.Landsize.mean())
df['BuildingArea'] = df['BuildingArea'].fillna(df.BuildingArea.mean())

df.isna().sum()

Suburb              0
Rooms               0
Type                0
Method              0
SellerG             0
Regionname          3
Propertycount       0
Distance            0
CouncilArea         3
Bedroom2            0
Bathroom            0
Car                 0
Landsize            0
BuildingArea        0
Price            7610
dtype: int64

In [9]:
df.dropna(inplace=True)
df

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.000000,160.2564,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.000000,79.0000,1035000.0
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.000000,150.0000,1465000.0
5,Abbotsford,3,h,PI,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,94.000000,160.2564,850000.0
6,Abbotsford,4,h,VB,Nelson,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,1.0,2.0,120.000000,142.0000,1600000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Yarraville,4,h,PI,Jas,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,4.0,1.0,3.0,593.000000,160.2564,1480000.0
34853,Yarraville,2,h,SP,Sweeney,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,2.0,2.0,1.0,98.000000,104.0000,888000.0
34854,Yarraville,2,t,S,Jas,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,2.0,1.0,2.0,220.000000,120.0000,705000.0
34855,Yarraville,3,h,SP,hockingstuart,Western Metropolitan,6543.0,6.3,Maribyrnong City Council,0.0,0.0,0.0,593.598993,160.2564,1140000.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27244 entries, 1 to 34856
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27244 non-null  object 
 1   Rooms          27244 non-null  int64  
 2   Type           27244 non-null  object 
 3   Method         27244 non-null  object 
 4   SellerG        27244 non-null  object 
 5   Regionname     27244 non-null  object 
 6   Propertycount  27244 non-null  float64
 7   Distance       27244 non-null  float64
 8   CouncilArea    27244 non-null  object 
 9   Bedroom2       27244 non-null  float64
 10  Bathroom       27244 non-null  float64
 11  Car            27244 non-null  float64
 12  Landsize       27244 non-null  float64
 13  BuildingArea   27244 non-null  float64
 14  Price          27244 non-null  float64
dtypes: float64(8), int64(1), object(6)
memory usage: 3.3+ MB


In [11]:
df.nunique()

Suburb            344
Rooms              12
Type                3
Method              5
SellerG           349
Regionname          8
Propertycount     336
Distance          213
CouncilArea        33
Bedroom2           14
Bathroom           10
Car                13
Landsize         1558
BuildingArea      663
Price            2871
dtype: int64

In [12]:
# Suburb, Type, Method, SellerG, Regionname, CouncilArea
344 + 3 + 5 + 349 + 8 + 33 - 6 + 15 - 6

745

In [13]:
df = pd.get_dummies(df,drop_first=True,dtype='int16')
df

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,Suburb_Airport West,Suburb_Albanvale,Suburb_Albert Park,Suburb_Albion,Suburb_Alphington,Suburb_Altona,Suburb_Altona Meadows,Suburb_Altona North,Suburb_Ardeer,Suburb_Armadale,Suburb_Ascot Vale,Suburb_Ashburton,Suburb_Ashwood,Suburb_Aspendale,Suburb_Aspendale Gardens,Suburb_Attwood,Suburb_Avondale Heights,Suburb_Bacchus Marsh,Suburb_Balaclava,Suburb_Balwyn,Suburb_Balwyn North,Suburb_Bayswater,Suburb_Bayswater North,Suburb_Beaconsfield,Suburb_Beaconsfield Upper,Suburb_Beaumaris,Suburb_Bellfield,Suburb_Bentleigh,Suburb_Bentleigh East,Suburb_Berwick,Suburb_Black Rock,Suburb_Blackburn,Suburb_Blackburn North,Suburb_Blackburn South,Suburb_Bonbeach,Suburb_Boronia,Suburb_Botanic Ridge,Suburb_Box Hill,Suburb_Braybrook,Suburb_Briar Hill,Suburb_Brighton,Suburb_Brighton East,Suburb_Broadmeadows,Suburb_Brookfield,Suburb_Brooklyn,Suburb_Brunswick,Suburb_Brunswick East,Suburb_Brunswick West,Suburb_Bulla,Suburb_Bulleen,Suburb_Bullengarook,Suburb_Bundoora,Suburb_Burnley,Suburb_Burnside,Suburb_Burnside Heights,Suburb_Burwood,Suburb_Burwood East,Suburb_Cairnlea,Suburb_Camberwell,Suburb_Campbellfield,Suburb_Canterbury,Suburb_Carlton,Suburb_Carlton North,Suburb_Carnegie,Suburb_Caroline Springs,Suburb_Carrum,Suburb_Carrum Downs,Suburb_Caulfield,Suburb_Caulfield East,Suburb_Caulfield North,Suburb_Caulfield South,Suburb_Chadstone,Suburb_Chelsea,Suburb_Chelsea Heights,Suburb_Cheltenham,Suburb_Chirnside Park,Suburb_Clarinda,Suburb_Clayton,Suburb_Clayton South,Suburb_Clifton Hill,Suburb_Clyde North,Suburb_Coburg,Suburb_Coburg North,Suburb_Coldstream,Suburb_Collingwood,Suburb_Coolaroo,Suburb_Craigieburn,Suburb_Cranbourne,Suburb_Cranbourne East,Suburb_Cranbourne North,Suburb_Cranbourne West,Suburb_Cremorne,Suburb_Croydon,Suburb_Croydon Hills,Suburb_Croydon North,Suburb_Croydon South,Suburb_Dallas,Suburb_Dandenong,Suburb_Dandenong North,Suburb_Darley,Suburb_Deepdene,Suburb_Deer Park,Suburb_Delahey,Suburb_Derrimut,Suburb_Diamond Creek,Suburb_Diggers Rest,Suburb_Dingley Village,Suburb_Docklands,Suburb_Doncaster,Suburb_Doncaster East,Suburb_Donvale,Suburb_Doreen,Suburb_Doveton,Suburb_Eaglemont,Suburb_East Melbourne,Suburb_Edithvale,Suburb_Elsternwick,Suburb_Eltham,Suburb_Eltham North,Suburb_Elwood,Suburb_Emerald,Suburb_Endeavour Hills,Suburb_Epping,Suburb_Essendon,Suburb_Essendon North,Suburb_Essendon West,Suburb_Eumemmerring,Suburb_Eynesbury,Suburb_Fairfield,Suburb_Fawkner,Suburb_Ferntree Gully,Suburb_Ferny Creek,Suburb_Fitzroy,Suburb_Fitzroy North,Suburb_Flemington,Suburb_Footscray,Suburb_Forest Hill,Suburb_Frankston,Suburb_Frankston North,Suburb_Frankston South,Suburb_Gardenvale,Suburb_Gisborne,Suburb_Gisborne South,Suburb_Gladstone Park,Suburb_Glen Huntly,Suburb_Glen Iris,Suburb_Glen Waverley,Suburb_Glenroy,Suburb_Gowanbrae,Suburb_Greensborough,Suburb_Greenvale,Suburb_Hadfield,Suburb_Hallam,Suburb_Hampton,Suburb_Hampton East,Suburb_Hampton Park,Suburb_Hawthorn,Suburb_Hawthorn East,Suburb_Healesville,Suburb_Heatherton,Suburb_Heathmont,Suburb_Heidelberg,Suburb_Heidelberg Heights,Suburb_Heidelberg West,Suburb_Highett,Suburb_Hillside,Suburb_Hoppers Crossing,Suburb_Hughesdale,Suburb_Huntingdale,Suburb_Hurstbridge,Suburb_Ivanhoe,Suburb_Ivanhoe East,Suburb_Jacana,Suburb_Kalkallo,Suburb_Kealba,Suburb_Keilor,Suburb_Keilor Downs,Suburb_Keilor East,Suburb_Keilor Lodge,Suburb_Keilor Park,Suburb_Kensington,Suburb_Kew,Suburb_Kew East,Suburb_Keysborough,Suburb_Kilsyth,Suburb_Kings Park,Suburb_Kingsbury,Suburb_Kingsville,Suburb_Knoxfield,Suburb_Kooyong,Suburb_Kurunjang,Suburb_Lalor,Suburb_Langwarrin,Suburb_Laverton,Suburb_Lilydale,Suburb_Lower Plenty,Suburb_Lynbrook,Suburb_Lysterfield,Suburb_MacLeod,Suburb_Maidstone,Suburb_Malvern,Suburb_Malvern East,Suburb_Maribyrnong,Suburb_McKinnon,Suburb_Meadow Heights,Suburb_Melbourne,Suburb_Melton,Suburb_Melton South,Suburb_Melton West,Suburb_Mentone,Suburb_Mernda,Suburb_Mickleham,Suburb_Middle Park,Suburb_Mill Park,Suburb_Mitcham,Suburb

In [14]:
X = df.drop('Price', axis=1).values
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7,random_state=2)
print(len(X_train))
print(len(X_test))

19070
8174


# Linear

In [15]:
lr = LinearRegression()
lr.fit(X_train,y_train)

Yp_train = lr.predict(X_train)
Yp_test = lr.predict(X_test)

In [16]:
print(lr.coef_.shape)
lr.coef_.round(0)

(744,)


array([ 2.6400200e+05,  5.0000000e+00, -4.6473000e+04, -8.2235000e+04,
        1.1715100e+05,  4.2970000e+04,  2.0000000e+00,  4.7000000e+02,
        2.6134200e+05, -4.6101000e+04, -1.3231200e+05,  2.1545400e+05,
        1.2013400e+05,  2.7268400e+05,  1.7470200e+05, -8.0020000e+04,
       -1.5096800e+05, -4.9446000e+04,  1.1771100e+05, -1.2888000e+05,
       -3.4935000e+04, -8.5430000e+03,  6.1916000e+04, -2.5725800e+05,
       -1.1086800e+05, -2.4239200e+05,  1.7985700e+05, -1.2239500e+05,
        2.0462500e+05,  3.7826000e+04,  1.4668100e+05,  2.3220000e+03,
        7.3540000e+04, -5.2920000e+04,  1.9048200e+05, -3.2045200e+05,
        7.4952000e+04, -2.8302000e+04,  2.6910000e+04,  2.7882400e+05,
        1.5138700e+05, -1.0632400e+05, -8.4509000e+04,  2.9656800e+05,
        1.5064500e+05, -0.0000000e+00,  1.7600000e+05, -1.2005700e+05,
       -9.6450000e+03,  3.6837700e+05, -1.3547900e+05, -6.3809000e+04,
        3.5349000e+04,  1.1236500e+05,  1.5021000e+04,  3.8070000e+04,
      

In [17]:
lr.intercept_

636394.7291872636

In [18]:
print(lr.score(X_train,y_train))
print(lr.score(X_test,y_test))

0.6827792395792723
0.1385368316171096


In [19]:
print(mean_squared_error(y_train, Yp_train))
print(mean_squared_error(y_test, Yp_test))

132454004507.99455
342275472103.4609


# Lasso (L1)

In [20]:
lasso = Lasso(alpha=50, max_iter=100, tol=0.1)
lasso.fit(X_train,y_train)

Yp_train = lasso.predict(X_train)
Yp_test = lasso.predict(X_test)

In [21]:
print(lasso.coef_.shape)
lasso.coef_.round(0)

(744,)


array([ 2.709680e+05,  5.000000e+00, -3.033600e+04, -8.489500e+04,
        1.232200e+05,  4.175000e+04,  3.000000e+00,  8.700000e+01,
        2.192210e+05, -1.200660e+05, -0.000000e+00,  3.240950e+05,
        1.554850e+05,  2.970720e+05,  1.381400e+05, -9.412500e+04,
       -1.301210e+05, -0.000000e+00,  1.516470e+05, -9.605000e+04,
       -1.743300e+04, -0.000000e+00,  2.296500e+04, -1.429070e+05,
       -0.000000e+00, -2.722950e+05,  0.000000e+00, -3.515800e+04,
        2.310860e+05,  6.082600e+04,  3.207500e+04, -0.000000e+00,
        0.000000e+00, -0.000000e+00,  1.901550e+05, -1.904160e+05,
        3.358800e+04, -8.788600e+04,  5.126600e+04,  2.576290e+05,
        1.863430e+05, -5.533000e+03, -0.000000e+00,  6.298800e+04,
        1.707200e+04,  0.000000e+00,  2.460770e+05, -1.208830e+05,
       -0.000000e+00,  4.887630e+05, -0.000000e+00,  1.274300e+04,
        0.000000e+00,  1.927770e+05,  3.586700e+04,  6.420200e+04,
       -0.000000e+00,  0.000000e+00, -1.735510e+05,  0.000000e

In [22]:
lasso.intercept_

559404.2311875502

In [23]:
print(lasso.score(X_train,y_train))
print(lasso.score(X_test,y_test))

0.6766985624766824
0.6636111369404488


In [24]:
print(mean_squared_error(y_train, Yp_train))
print(mean_squared_error(y_test, Yp_test))

134992962019.1294
133653603705.64287


# Ridge (L2)

In [25]:
ridge = Ridge(alpha=50, max_iter=100, tol=0.1)
ridge.fit(X_train,y_train)

Yp_train = ridge.predict(X_train)
Yp_test = ridge.predict(X_test)

In [26]:
print(ridge.coef_.shape)
ridge.coef_.round(0)

(744,)


array([ 2.74565e+05,  1.00000e+00, -3.08680e+04, -8.54800e+04,
        1.30784e+05,  3.79900e+04,  3.00000e+00,  3.50000e+01,
        1.28807e+05, -6.43940e+04, -2.20190e+04,  1.41226e+05,
        6.01540e+04,  1.36605e+05,  6.82450e+04, -4.48760e+04,
       -1.18261e+05, -1.40380e+04,  9.47700e+04, -5.29140e+04,
       -4.41700e+04, -4.98010e+04,  2.40850e+04, -3.83940e+04,
       -1.58350e+04, -1.68202e+05,  8.08500e+03, -6.27880e+04,
        1.48802e+05,  3.01080e+04,  2.59810e+04, -1.38080e+04,
        1.17010e+04,  9.42000e+02,  7.84150e+04, -6.16680e+04,
        3.86960e+04, -5.72490e+04,  4.67430e+04,  6.78010e+04,
        7.05290e+04, -3.93760e+04, -2.74950e+04,  2.49850e+04,
        2.99160e+04,  0.00000e+00,  1.14985e+05, -7.42920e+04,
       -1.36010e+04,  3.69879e+05, -4.72150e+04, -8.03300e+03,
        2.80900e+03,  5.43350e+04,  5.11640e+04,  4.72230e+04,
        2.35100e+03,  0.00000e+00, -9.56250e+04,  2.17240e+04,
       -1.35131e+05, -8.56700e+03, -7.43700e+03, -2.836

In [27]:
ridge.intercept_

624628.4741325371

In [28]:
print(ridge.score(X_train,y_train))
print(ridge.score(X_test,y_test))

0.6622376739684328
0.6670848945194958


In [29]:
print(mean_squared_error(y_train, Yp_train))
print(mean_squared_error(y_test, Yp_test))

141031036542.1237
132273414674.95778
